In [5]:
# !pip install git+https://github.com/albumentations-team/albumentations.git
!pip install --upgrade --force-reinstall --no-deps albumentations


  Using cached https://files.pythonhosted.org/packages/03/58/63fb1d742dc42d9ba2800ea741de1f2bc6bb05548d8724aa84794042eaf2/albumentations-0.5.2-py3-none-any.whl
  Found existing installation: albumentations 0.5.2
    Uninstalling albumentations-0.5.2:
      Successfully uninstalled albumentations-0.5.2


In [6]:
# !pip show torch

In [7]:
import torch
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from PIL import Image
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
from tqdm import tqdm


DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 2e-4
BATCH_SIZE = 16
NUM_WORKERS = 2
IMG_SIZE = 256
CH_IMG = 3
L1_LAMBDA = 100
NUM_EPOCHS = 500
LOAD_MODEL = False
SAVE_MODEL = True
CHECKPOINT_DISC = "disc.pth.tar"
CHECKPOINT_GEN = "gen.pth.tar"
TRAIN_PATH = "/content/maps/train"
VAL_PATH = "/content/maps/val"
EVAL_PATH = "/content/maps/evaluation"

both_transform = A.Compose(
    [A.Resize(width=256, height=256)], additional_targets={"image0": "image"}
)

transform_only_input = A.Compose(
    [
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0),
        A.pytorch.transforms.ToTensor(),
    ]
)

transform_only_mask = A.Compose(
    [
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0),
        A.pytorch.transforms.ToTensor(),
    ]
)

class MapDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.list_files = os.listdir(self.root_dir)

    def __len__(self):
        return len(self.list_files)
    
    def __getitem__(self, index):
        img_name = self.list_files[index]
        img_path = os.path.join(self.root_dir, img_name)
        print(img_path)
        image = np.array(Image.open(img_path))
        input_image = image[:, :600, :]
        target_image = image[:, 600:, :]

        augmentations = both_transform(image=input_image, image0=target_image)
        input_image, target_image = augmentations["image"], augmentations["image0"]

        input_image = transform_only_input(image=input_image)["image"]
        target_image = transform_only_mask(image=target_image)["image"]
        return input_image, target_image

def save_some_examples(gen, val_loader, epoch, folder):
    x, y = next(iter(val_loader))
    x, y = x.to(DEVICE), y.to(DEVICE)
    gen.eval()
    if not os.path.exists(folder): os.mkdir(folder) 
    with torch.no_grad():
        y_fake = gen(x)
        y_fake = y_fake * 0.5 + 0.5
        save_image(y_fake, folder + f"/y_gen_{epoch}.png")
        save_image(x*0.5 + 0.5, folder + f"/input_{epoch}.png")
        if epoch == 1:
            save_image(y*0.5 + 0.5, folder + f"/label_{epoch}.png")
    gen.train()

def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
    torch.save(checkpoint, filename)
def load_checkpoint(checkpoint_file, model, optimizer, lr):
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    for param_group in optimizer.param_groups:
        param_groups["lr"] = lr

In [8]:
import torch.nn as nn

class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=2):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=4, stride=stride, bias=False, padding_mode="reflect"),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.conv(x)

class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels*2, features[0], kernel_size=4, stride=2, padding=1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )
        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(
                CNNBlock(in_channels, feature, stride=1 if feature == features[-1] else 2)
            )
            in_channels = feature
        layers.append(
            nn.Conv2d(
                in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"
            )
        )
        self.model = nn.Sequential(*layers)
    
    def forward(self, x, y):
        x = torch.cat([x,y], dim=1)
        x = self.initial(x)
        return self.model(x)

def test_Disc():
    x = torch.randn((1, 3, 286, 286))
    y = torch.randn((1, 3, 286, 286))
    model = Discriminator()
    preds = model(x, y)
    print(preds.shape)

test_Disc()

torch.Size([1, 1, 30, 30])


In [9]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act="relu", use_dropout=False):
        super().__init__()
        self.conv = nn.Sequential(
            nn. Conv2d(in_channels, out_channels, 4, 2, 1, bias=False, padding_mode="reflect")
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act=="relu" else nn.LeakyReLU(0.2),
        )
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv(x)
        return self.dropout(x) if self.use_dropout else x

class Generator(nn.Module):
    def __init__(self, in_channels=3, features=64):
        super().__init__()
        self.initial_down = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        ) # 128
        self.down1 = Block(features, features*2, down=True, act="Leaky", use_dropout=False) # 64
        self.down2 = Block(features*2, features*4, down=True, act="Leaky", use_dropout=False) # 32
        self.down3 = Block(features*4, features*8, down=True, act="Leaky", use_dropout=False) # 16
        self.down4 = Block(features*8, features*8, down=True, act="Leaky", use_dropout=False) # 8
        self.down5 = Block(features*8, features*8, down=True, act="Leaky", use_dropout=False) # 4
        self.down6 = Block(features*8, features*8, down=True, act="Leaky", use_dropout=False) # 2
        self.bottleneck = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1), nn.ReLU(), # 1x1
        )
        self.up1 = Block(features*8, features*8, down=False, act="relu", use_dropout=True)
        self.up2 = Block(features*8*2, features*8, down=False, act="relu", use_dropout=True)
        self.up3 = Block(features*8*2, features*8, down=False, act="relu", use_dropout=True)
        self.up4 = Block(features*8*2, features*8, down=False, act="relu", use_dropout=False)
        self.up5 = Block(features*8*2, features*4, down=False, act="relu", use_dropout=False)
        self.up6 = Block(features*4*2, features*2, down=False, act="relu", use_dropout=False)
        self.up7 = Block(features*2*2, features, down=False, act="relu", use_dropout=False)
        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(features*2, in_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        d1 = self.initial_down(x)
        d2 = self.down1(d1)
        d3 = self.down2(d2)
        d4 = self.down3(d3)
        d5 = self.down4(d4)
        d6 = self.down5(d5)
        d7 = self.down6(d6)
        bottleneck = self.bottleneck(d7)
        up1 = self.up1(bottleneck)
        up2 = self.up2(torch.cat([up1, d7], 1))
        up3 = self.up3(torch.cat([up2, d6], 1))
        up4 = self.up4(torch.cat([up3, d5], 1))
        up5 = self.up5(torch.cat([up4, d4], 1))
        up6 = self.up6(torch.cat([up5, d3], 1))
        up7 = self.up7(torch.cat([up6, d2], 1))
        return self.final_up(torch.cat([up7, d1], 1))

def test_gen():
     x = torch.randn((1, 3, 256, 256))
     model = Generator(in_channels=3, features=64)    
     preds = model(x)
     print(preds.shape)    

test_gen()
    


torch.Size([1, 3, 256, 256])


In [10]:
import torch.optim as optim

def train_fn(disc, gen, loader, opt_disc, opt_gen, l1, bce, g_scaler, d_scaler):
    loop = tqdm(loader, leave=True)

    for idx, (x, y) in enumerate(loop):
        x, y = x.to(DEVICE), y.to(DEVICE)

        # Train Discriminator
        with torch.cuda.amp.autocast(): # float16 training for less VRAM
            y_fake = gen(x)
            D_real = disc(x, y)
            D_fake = disc(x, y_fake.detach())
            D_real_loss = bce(D_real, torch.ones_like(D_real))
            D_fake_loss = bce(D_fake, torch.zeros_like(D_fake))
            D_loss = (D_real_loss + D_fake_loss) / 2

        disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train Generator
        with torch.cuda.amp.autocast():
            D_fake = disc(x, y_fake)
            G_fake_loss = bce(D_fake, torch.ones_like(D_fake))
            L1 = l1(y_fake, y) * L1_LAMBDA
            G_loss = G_fake_loss + L1

        gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

def main():
    disc = Discriminator(in_channels=3).to(DEVICE)
    gen = Generator(in_channels=3). to(DEVICE)
    opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
    opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
    BCE = nn.BCEWithLogitsLoss()
    L1_LOSS = nn.L1Loss()

    if LOAD_MODEL:
        load_checkpoint(CHECKPOINT_GEN, gen, opt_gen, LEARNING_RATE)
        load_checkpoint(CHECKPOINT_DISC, disc, opt_disc, LEARNING_RATE)

    train_dataset = MapDataset(root_dir=TRAIN_PATH)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    g_scaler = torch.cuda.amp.GradScaler() # float16 training for less VRAM
    d_scaler = torch.cuda.amp.GradScaler()
    val_dataset = MapDataset(root_dir=VAL_PATH)
    val_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
    
    for epoch in range(NUM_EPOCHS):
        train_fn(disc, gen, train_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler)
        
        if SAVE_MODEL and epoch % 5 == 0:
            save_checkpoint(gen, opt_gen, filename=CHECKPOINT_GEN) 
            save_checkpoint(disc, opt_disc, filename=CHECKPOINT_DISC)
        save_some_examples(gen, val_loader, epoch, folder=EVAL_PATH)     

In [ ]:
import tarfile
import urllib.request

if not os.path.exists("/content/maps/"):
    #"https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/maps.tar.gz"
    urllib.request.urlretrieve ("https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/maps.tar.gz"
    , "maps.tar.gz")

    tar = tarfile.open("maps.tar.gz", "r:gz")
    tar.extractall()
    tar.close()
main()

  0%|          | 0/69 [00:00<?, ?it/s]

/content/maps/train/543.jpg
/content/maps/train/308.jpg
/content/maps/train/318.jpg
/content/maps/train/788.jpg
/content/maps/train/39.jpg
/content/maps/train/238.jpg
/content/maps/train/641.jpg
/content/maps/train/138.jpg
/content/maps/train/154.jpg
/content/maps/train/292.jpg
/content/maps/train/616.jpg
/content/maps/train/528.jpg
/content/maps/train/523.jpg
/content/maps/train/324.jpg
/content/maps/train/545.jpg
/content/maps/train/807.jpg
/content/maps/train/861.jpg
/content/maps/train/484.jpg
/content/maps/train/834.jpg
/content/maps/train/329.jpg
/content/maps/train/692.jpg
/content/maps/train/51.jpg
/content/maps/train/606.jpg
/content/maps/train/145.jpg
/content/maps/train/806.jpg
/content/maps/train/1005.jpg
/content/maps/train/802.jpg
/content/maps/train/597.jpg
/content/maps/train/968.jpg
/content/maps/train/250.jpg
/content/maps/train/1010.jpg
/content/maps/train/101.jpg
/content/maps/train/139.jpg
/content/maps/train/9.jpg
/content/maps/train/869.jpg
/content/maps/train/84

  1%|▏         | 1/69 [00:01<01:08,  1.00s/it]

/content/maps/train/1083.jpg
/content/maps/train/815.jpg
/content/maps/train/376.jpg
/content/maps/train/872.jpg
/content/maps/train/386.jpg
/content/maps/train/515.jpg
/content/maps/train/259.jpg
/content/maps/train/470.jpg
/content/maps/train/50.jpg
/content/maps/train/919.jpg
/content/maps/train/327.jpg
/content/maps/train/1046.jpg


  3%|▎         | 2/69 [00:01<00:51,  1.29it/s]

...

/content/maps/train/543.jpg
/content/maps/train/610.jpg
/content/maps/train/118.jpg
/content/maps/train/748.jpg
/content/maps/train/99.jpg
/content/maps/train/502.jpg
/content/maps/train/492.jpg
/content/maps/train/415.jpg
/content/maps/train/869.jpg
/content/maps/train/1031.jpg
/content/maps/train/448.jpg
/content/maps/train/398.jpg
/content/maps/train/418.jpg
/content/maps/train/65.jpg


 81%|████████  | 56/69 [00:34<00:08,  1.60it/s]

/content/maps/train/935.jpg
/content/maps/train/557.jpg
/content/maps/train/719.jpg
/content/maps/train/954.jpg
/content/maps/train/676.jpg
/content/maps/train/250.jpg
/content/maps/train/468.jpg
/content/maps/train/758.jpg
/content/maps/train/204.jpg
/content/maps/train/753.jpg
/content/maps/train/788.jpg
/content/maps/train/211.jpg
/content/maps/train/768.jpg
/content/maps/train/899.jpg
/content/maps/train/585.jpg
/content/maps/train/180.jpg
/content/maps/train/323.jpg
/content/maps/train/98.jpg
/content/maps/train/34.jpg


 83%|████████▎ | 57/69 [00:35<00:06,  1.76it/s]

/content/maps/train/68.jpg
/content/maps/train/652.jpg
/content/maps/train/831.jpg
/content/maps/train/626.jpg
/content/maps/train/1011.jpg
/content/maps/train/581.jpg
/content/maps/train/96.jpg
/content/maps/train/1027.jpg
/content/maps/train/414.jpg
/content/maps/train/445.jpg
/content/maps/train/475.jpg
/content/maps/train/62.jpg
/content/maps/train/826.jpg
/content/maps/train/179.jpg
/content/maps/train/144.jpg
